In [ ]:
import json
import pandas as pd


In [1]:
import os
import pandas as pd
import json

# Define paths
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/HiVerilog_expansion_eval/conversations.jsonl'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/HiVerilog_expansion_eval/graphs.jsonl'
tb_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/HiVerilog_expansion_eval/testbenches.jsonl'
target_dir = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/HiVerilog_expansion_eval/HiVerilog_benchmark_expansion'

# Load data
conv = pd.read_json(conv_path, lines=True)
graph = pd.read_json(graph_path, lines=True)
tb = pd.read_json(tb_path, lines=True)


In [5]:
type(tb.iloc[0]['testbench'][0])

str

In [2]:

# Ensure target directory exists
os.makedirs(target_dir, exist_ok=True)

# Iterate through conversations
for i in range(len(conv)):
    # Create subdirectory
    task_id = conv.iloc[i]["task_id"]
    sub_dir = os.path.join(target_dir, task_id)
    os.makedirs(sub_dir, exist_ok=True)

    # Create design_description.txt
    with open(os.path.join(sub_dir, "design_description.txt"), "w") as f:
        f.write("Please act as a professional Verilog designer.\n" + conv.iloc[i]["Instruction"])

    # Create graph.jsonl
    with open(os.path.join(sub_dir, "graph.jsonl"), "w") as f:
        json.dump(graph.iloc[i].to_dict(), f)

    # Create makefile
    makefile_content = f""".PHONY: vcs sim clean

TEST_DESIGN = {task_id}

vcs:
    vcs -sverilog +v2k -timescale=1ns/1ns       \\
    -debug_all         							\\
    -l compile.log 								\\
    ${{TEST_DESIGN}}.v testbench.v

sim:
    ./simv -l run.log

clean:
    rm -rf *.log  csrc  simv*  *.key *.vpd  DVEfiles coverage *.vdb output.txt"""
    with open(os.path.join(sub_dir, "makefile"), "w") as f:
        f.write(makefile_content)

    # Create testbench.v
    with open(os.path.join(sub_dir, "testbench.v"), "w") as f:
        f.write(tb.iloc[i]["testbench"][0])

    # Create verified_{task_id}.v
    with open(os.path.join(sub_dir, f"verified_{task_id}.v"), "w") as f:
        f.write(conv.iloc[i]["Response"])

print("Processing complete!")

Processing complete!
